# **Scraping Berita CNBC Indonesia**

## Importing Library

1. `from bs4 import BeautifulSoup`: Pustaka BeautifulSoup adalah pustaka Python yang digunakan untuk melakukan web scraping. Dalam konteks ini, BeautifulSoup akan digunakan untuk mengurai atau mem-parsing data HTML yang diambil dari situs web.

2. `import requests`: Pustaka `requests` adalah pustaka yang digunakan untuk membuat permintaan HTTP ke server web. Dalam kode ini, kita akan menggunakannya untuk mengunduh halaman web atau mengambil data dari situs web.

3. `import pandas as pd`: Pustaka `pandas` adalah pustaka Python yang digunakan untuk analisis data. Dalam kode ini, kita akan menggunakan pandas untuk menyimpan dan mengelola data yang diambil dari situs web ke dalam bentuk DataFrame, yang merupakan struktur data yang kuat untuk menganalisis dan memanipulasi data.

Dengan mengimpor pustaka-pustaka ini, kita akan siap untuk melakukan web scraping dan menganalisis data yang diambil dari situs web. Selanjutnya, Anda akan perlu menambahkan sel-sel kode tambahan untuk melakukan langkah-langkah berikutnya, seperti mengambil data dari situs web dan memprosesnya menggunakan pustaka-pustaka ini.

In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

## Function `get_nav_link()`

In [ ]:
def get_nav_link():
  response = requests.get('https://www.cnbcindonesia.com')

  # Untuk mengecek respon url dan mencari menu sidebar
  if response.status_code == 200:
    # Get Nav
    html = response.content
    nav = BeautifulSoup(html, 'html.parser').find_all('a', class_=lambda value: value and 'gtm_navbar' in value)
    nav_link = {i.text.strip().replace('\xa0', ' ') : i.get('href') for i in nav if i.text.strip().replace('\xa0', ' ') not in ['HOME', 'CUAP CUAP CUAN', 'RESEARCH', 'OPINI', 'FOTO', 'VIDEO', 'INFOGRAFIS', 'BERBUATBAIK.ID']}

    # Update Nav Link
    for category, link in nav_link.items():
      nav_content = requests.get(link).content
      new_link = BeautifulSoup(nav_content, 'html.parser').find('a', class_='btn btn_inline gtm_button_indeks_newsfeed').get('href')
      nav_link[category] = new_link

    return nav_link

  else:
    print(f'Response https://www.cnbcindonesia.com : {response.status_code}')

Kode di atas adalah definisi dari sebuah fungsi bernama `get_nav_link`. Fungsi ini dirancang untuk melakukan beberapa tugas berikut:

1. Mengambil halaman web dari situs web "https://www.cnbcindonesia.com".
2. Menganalisis halaman web untuk menemukan tautan navigasi (nav) ke berbagai kategori.
3. Memperbarui tautan kategori dengan tautan ke halaman berita terkait.
4. Mengembalikan kamus (dictionary) yang berisi tautan kategori sebagai kunci dan tautan halaman berita terkait sebagai nilainya.



## Function `get_data()`

In [ ]:
def get_data(url):
  response = requests.get(url)
  if response.status_code == 200:
    data = []
    html = response.content
    content = BeautifulSoup(html, 'html.parser')
    judul = content.find('h1').text.strip()
    date = content.find('div', class_='date').text.strip()
    category, author = content.find('div', class_='author').text.split('-')
    article = content.find('div', class_='detail_text').find_all('p')
    text = ''
    for i in article:
      if i.text.strip() not in ['SCROLL TO RESUME CONTENT', 'ADVERTISEMENT']:
        text += i.text.strip() + ' '
    data.append(url)
    data.append(judul)
    data.append(date)
    data.append(author.strip())
    data.append(text.replace('\xa0', ''))
    data.append(category.strip())
    return data

Kode di atas adalah definisi dari sebuah fungsi yang bernama `get_data`. Fungsi ini digunakan untuk mengambil dan menguraikan data dari halaman web berita dengan URL yang diberikan. Berikut adalah penjelasan singkat tentang apa yang dilakukan oleh fungsi ini:

- Nama Fungsi: `get_data`
- Parameter: `url` (URL halaman web berita yang akan diambil data dari)

Fungsi `get_data` melakukan langkah-langkah berikut:

1. Menggunakan pustaka `requests`, fungsi ini mengambil halaman web dari URL yang diberikan (`url`).

2. Memeriksa apakah responsenya berhasil (status kode 200), jika ya, maka melakukan langkah selanjutnya.

3. Menginisialisasi sebuah daftar kosong `data` yang akan digunakan untuk menyimpan data dari halaman web.

4. Mengurai kode HTML halaman web menggunakan BeautifulSoup.

5. Mengambil judul berita dari elemen `<h1>` pada halaman dan membersihkan teksnya.

6. Mengambil tanggal berita dari elemen dengan kelas CSS "date" dan membersihkan teksnya.

7. Mengambil kategori dan penulis berita dari elemen dengan kelas CSS "author" dan memisahkan kategori dan penulis menggunakan tanda '-'. Kemudian membersihkan teksnya.

8. Mengambil teks berita dari elemen dengan kelas CSS "detail_text" dan menghapus elemen-elemen yang tidak dibutuhkan seperti "SCROLL TO RESUME CONTENT" dan "ADVERTISEMENT". Teks berita kemudian disatukan ke dalam variabel `text`.

9. Data berita, termasuk URL, judul, tanggal, penulis, teks berita, dan kategori, ditambahkan ke dalam daftar `data`.

10. Fungsi mengembalikan `data`, yang berisi semua informasi yang telah diambil dari halaman web.

Jadi, fungsi `get_data` ini digunakan untuk mengambil dan menyajikan data yang relevan dari halaman web berita yang diberikan.

## Function `get_content()`

In [ ]:
def get_content(url, len_data, target):
  data = []
  response = requests.get(url)
  if response.status_code == 200:
    html = response.content
    container = BeautifulSoup(html, 'html.parser')
    list_content = container.find_all('article')
    next_button = container.find('div', class_='paging text_center gtm_paging').find_all('a')[-1].get('href')
    idx = 0

    # for content in list_content:
    while idx != len(list_content):
      if len_data != target:
        subjudul = list_content[idx].find('span', class_='subjudul')
        if subjudul == None or subjudul.text.strip() not in ["video","Breaking News","Techzone", "Video", "Editor's View","Foto Internasional" , "FOTO Internasional", "FOTO", "Video Breaking News", "Transmart Full Day Sale", "Tech A Look"]:
          data_berita = get_data(list_content[idx].find('a').get('href'))
          data.append(data_berita)
          len_data += 1
          print(data_berita)
      else:
        break
        return (data, next_button)
      idx += 1

  else:
    print(f'Response {url} : {response.status_code}')
  return (data, next_button)

Kode di atas adalah definisi dari sebuah fungsi yang bernama `get_content`. Fungsi ini digunakan untuk mengambil dan mengurai konten berita dari halaman web dengan URL yang diberikan. Berikut adalah penjelasan singkat tentang apa yang dilakukan oleh fungsi ini:

- Nama Fungsi: `get_content`
- Parameter: `url` (URL halaman web berita), `len_data` (panjang data yang sudah diambil), `target` (target panjang data yang ingin dicapai)

Fungsi `get_content` melakukan langkah-langkah berikut:

1. Menginisialisasi daftar kosong `data` yang akan digunakan untuk menyimpan data konten berita.

2. Menggunakan pustaka `requests`, fungsi ini mengambil halaman web dari URL yang diberikan (`url`).

3. Memeriksa apakah responsenya berhasil (status kode 200), jika ya, maka melakukan langkah selanjutnya.

4. Mengurai kode HTML halaman web menggunakan BeautifulSoup.

5. Mengidentifikasi elemen-elemen `<article>` yang mengandung konten berita pada halaman web.

6. Mengidentifikasi tautan ke halaman berita berikutnya (jika ada) dengan mencari elemen dengan kelas CSS "paging text_center gtm_paging" dan mengambil tautan dari elemen terakhir dalam daftar tautan.

7. Mengeksekusi loop `while` untuk mengambil data konten berita dari setiap elemen `<article>` dalam daftar `list_content`. Loop ini berlanjut selama `len_data` belum mencapai `target`.

8. Dalam setiap iterasi, fungsi memeriksa apakah ada elemen `<span>` dengan kelas CSS "subjudul" yang berisi teks tertentu yang perlu diabaikan. Jika tidak ada atau teksnya tidak ada dalam daftar yang perlu diabaikan, maka data berita diambil dengan memanggil fungsi `get_data` pada tautan yang ditemukan dan data tersebut ditambahkan ke dalam daftar `data`.

9. Fungsi juga meng-update `len_data` setiap kali berhasil mengambil data berita.

10. Fungsi menghentikan loop dan mengembalikan hasil (yaitu data berita yang telah diambil dan tautan ke halaman berita berikutnya) ketika `len_data` mencapai `target`.

11. Jika ada masalah dalam mengambil respons dari URL, maka akan mencetak pesan yang menyatakan kode status respons yang diterima.

12. Fungsi mengembalikan sebuah tupel yang berisi daftar `data` dan tautan ke halaman berita berikutnya.

Jadi, fungsi `get_content` ini digunakan untuk mengambil dan mengurai konten berita dari halaman web, serta menghubungkan dengan halaman berita berikutnya hingga mencapai target panjang data yang diinginkan.

## Function `scraping_cnbc()`

In [ ]:
def scraping_cnbc(category=['TECH', 'NEWS'], num_data=2):
  category_link = get_nav_link()
  data = []

  for i in category:
    len_data_category = 0
    while len_data_category != num_data:
      new_data, next_button = get_content(category_link[i], len_data_category, num_data)
      data += new_data
      len_data_category += len(new_data)
      category_link[i] = next_button

  print(f'Berhasil Mengambil data sebanyak {len(data)} berita dengan {len(category)} Category')

  df = pd.DataFrame(data, columns=['Url', 'Judul', 'Tanggal', 'Author', 'Artikel', 'Category'])
  df.to_csv('Data Berita CNBC.csv', index=False)

Kode di atas adalah definisi dari sebuah fungsi yang bernama `scraping_cnbc`. Fungsi ini adalah fungsi utama yang digunakan untuk melakukan scraping berita dari situs web "CNBC Indonesia" dengan kategori dan jumlah data yang diinginkan. Berikut adalah penjelasan singkat tentang apa yang dilakukan oleh fungsi ini:

- Nama Fungsi: `scraping_cnbc`
- Parameter: `category` (kategori berita yang ingin diambil, default ['TECH', 'NEWS']), `num_data` (jumlah data berita yang ingin diambil, default 2)

Fungsi `scraping_cnbc` melakukan langkah-langkah berikut:

1. Memanggil fungsi `get_nav_link` untuk mendapatkan tautan kategori berita dari situs "CNBC Indonesia" dan menyimpannya dalam variabel `category_link`.

2. Membuat daftar kosong `data` yang akan digunakan untuk menyimpan data berita yang akan diambil.

3. Mengeksekusi loop `for` untuk setiap kategori yang diberikan dalam parameter `category`. Di dalam loop ini, fungsi akan mengambil data berita dari kategori yang sedang diproses.

4. Di dalam loop, fungsi menginisialisasi variabel `len_data_category` yang digunakan untuk melacak jumlah data berita yang telah diambil dari kategori tertentu.

5. Di dalam loop, fungsi memanggil `get_content` untuk mengambil data berita dari tautan kategori yang sesuai. Data berita yang baru diambil ditambahkan ke dalam daftar `data`.

6. Variabel `len_data_category` diperbarui dengan panjang data berita yang baru diambil.

7. Tautan ke halaman berita berikutnya (jika ada) juga diperbarui dalam variabel `category_link`.

8. Setelah mengambil data berita sesuai dengan jumlah yang diinginkan, loop untuk kategori berakhir.

9. Fungsi mencetak pesan yang menyatakan berapa banyak data berita yang berhasil diambil dari semua kategori.

10. Fungsi menggunakan pandas untuk membuat DataFrame dari data berita yang telah diambil dan menyimpannya dalam file CSV dengan nama "Data Berita CNBC.csv".

Jadi, fungsi `scraping_cnbc` ini adalah fungsi utama yang mengorganisir scraping data berita dari situs web "CNBC Indonesia" berdasarkan kategori dan jumlah data yang diinginkan, dan kemudian menyimpan data tersebut dalam sebuah file CSV.

## Pemanggilan Fungsi `scraping_cnbc()`

Pemanggilan fungsi `scraping_cnbc` dengan parameter `category=['TECH', 'NEWS', 'ENTREPRENEUR']` dan `num_data=1` akan menjalankan proses scraping berita dari situs web "CNBC Indonesia" untuk tiga kategori berita: 'TECH', 'NEWS', dan 'ENTREPRENEUR', dan hanya mengambil satu data berita dari setiap kategori. Hasil scraping akan disimpan dalam sebuah file CSV dengan nama "Data Berita CNBC.csv". Setelah proses selesai, fungsi akan mencetak pesan yang menyatakan berapa banyak data berita yang berhasil diambil.

In [ ]:
scraping_cnbc(category=['TECH', 'NEWS', 'ENTREPRENEUR'], num_data=1)

Output hidden; open in https://colab.research.google.com to view.

## Load Data Hasil Scraping

menggunakan pustaka pandas untuk membaca file CSV yang telah dihasilkan oleh fungsi `scraping_cnbc`. Setelah membaca file CSV, Anda menyimpan data tersebut dalam DataFrame yang dinamai `df`. Kemudian, Anda menampilkan DataFrame `df` dengan perintah `df`. Ini akan menampilkan data berita yang telah diambil dari situs web "CNBC Indonesia" dalam bentuk DataFrame di lingkungan kerja Anda. Anda dapat melakukan berbagai operasi dan analisis data lebih lanjut menggunakan DataFrame ini.

In [ ]:
df = pd.read_csv('Data Berita CNBC.csv')
df

,Url,Judul,Tanggal,Author,Artikel,Category
0,https://www.cnbcindonesia.com/tech/20231106180...,Astronom Yakin Alien Sudah Tahu di Bumi Ada Ma...,06 November 2023 19:50,"Intan Rakhmayanti Dewi, CNBC Indonesia","Jakarta, CNBC Indonesia - Astronom yakin alien...",Tech
1,https://www.cnbcindonesia.com/tech/20231106175...,Scarlett Johansson Ngamuk Wajah dan Suaranya D...,06 November 2023 19:00,"Intan Rakhmayanti Dewi, CNBC Indonesia","Jakarta, CNBC Indonesia - Aktrisnominasi Oscar...",Tech
2,https://www.cnbcindonesia.com/tech/20231106164...,"Dulu Bikin Ghozali Kaya Raya, Startup Ini Peca...",06 November 2023 18:20,"Novina Putri Bestari, CNBC Indonesia","Jakarta, CNBC Indonesia - Masih ingat Ghozali ...",Tech
3,https://www.cnbcindonesia.com/tech/20231106162...,"Jeff Bezos Pindah ke Rumah Rp 2,3 T Demi Pacar...",06 November 2023 17:40,"Novina Putri Bestari, CNBC Indonesia","Jakarta, CNBC Indonesia - Jeff Bezos memutuska...",Tech
4,https://www.cnbcindonesia.com/tech/20231106172...,BRIBRAIN Sukses Raih Penghargaan Ini di IDC Aw...,06 November 2023 17:33,"Khoirul Anam, CNBC Indonesia","Jakarta, CNBC Indonesia- PT Bank Rakyat Indone...",Tech
...,...,...,...,...,...,...
1495,https://www.cnbcindonesia.com/entrepreneur/202...,"Bisnis Kopi Meluas, Kompetisi Barista dan Latt...",03 April 2023 12:41,"Zefanya Aprilia, CNBC Indonesia","Jakarta, CNBCIndonesia -Roadshow Bezzera Latte...",Entrepreneur
1496,https://www.cnbcindonesia.com/entrepreneur/202...,"Rahusna, Alumni ITB Penemu Mesin Pemilah Sampa...",03 April 2023 12:33,"Mentari Puspadini, CNBC Indonesia","Jakarta, CNBC Indonesia - Hingga saat ini, per...",Entrepreneur
1497,https://www.cnbcindonesia.com/entrepreneur/202...,Pameran Barang Mewah Terbesar akan Hadir di RI...,31 March 2023 12:45,"Zefanya Aprilia, CNBC Indonesia","Jakarta, CNBCIndonesia -Indonesia akan menyele...",Entrepreneur
1498,https://www.cnbcindonesia.com/entrepreneur/202...,SETC & INOTEK Kerja Sama Gelar Pelatihan UMKM ...,31 March 2023 09:30,"Teti Purwanti, CNBC Indonesia","Jakarta, CNBC Indonesia - Sampoerna Entreprene...",Entrepreneur


## Cek Jumlah Data

In [ ]:
df.shape

(1500, 6)

## Cek Data NULL

In [ ]:
df.isnull().sum()

Url         0
Judul       0
Tanggal     0
Author      0
Artikel     0
Category    0
dtype: int64